# OpenAI Agent with LlamaIndex

## Install Dependencies

In [19]:
!pip install uv
!uv pip install -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv
!pip install -q -U google-genai


## Setup API Keys


In [15]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = environ["GEMINI_API_KEY"]

## Import libraries and setup LlamaIndex

In [30]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

#used for getting folder and file path for resume.
from pathlib import Path 

# # Create an llm object to use for the QueryEngine and the ReActAgent
# llm = OpenAI(model="gpt-4")
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

# response = client.models.generate_content(
#     model="gemini-2.5-flash", contents="Explain how AI works in a few words"
# )
# print(response.text)

Gets the resume file in the input resume directory

In [41]:
def GetFirstFile(directory_path):
    try:
        resume_folder = pathlib.Path(directory_path)
        for resume in resume_folder.iterdir():
            #only look for pdf files.
            if resume.is_file() and resume.suffix == ".pdf":
                return resume.absolute()
        return None
    except FileNotFoundError:
        return None

resume_path = GetFirstFile("../InputResume")
print(resume_path)


None


# Set up Phoenix

In [22]:
import phoenix as px
session = px.launch_app()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [24]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



ImportError: cannot import name 'ContentBlock' from 'llama_index.core.base.llms.types' (/home/lilbluey/repos/20k-project/.venv/lib/python3.12/site-packages/llama_index/core/base/llms/types.py)

## Load Documents

In [5]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

/Users/yujian/Documents/workspace/ai_agents_cookbooks/p312/lib/python3.12/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [6]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
    uber_index = VectorStoreIndex.from_documents(uber_docs, swow_progress=True)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

Now create the query engines.

In [7]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [9]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [11]:
response = agent.chat("Which do you like better? Lyft or Uber?")
print(str(response))

> Running step f096df50-a6f3-4889-b875-529dbed43324. Step input: Which do you like better? Lyft or Uber?
Thought: As an artificial intelligence, I don't have personal experiences or preferences. I can provide information and analysis based on data, but I don't have personal opinions or feelings.
Answer: As an artificial intelligence, I don't have personal preferences or feelings, so I don't have a preference between Lyft and Uber. I'm here to provide objective information and help answer your questions to the best of my ability.
As an artificial intelligence, I don't have personal preferences or feelings, so I don't have a preference between Lyft and Uber. I'm here to provide objective information and help answer your questions to the best of my ability.
